<a href="https://colab.research.google.com/github/SteinwayThor/eeg-features-movies/blob/main/EDA_and_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>